In [2]:
from typing_extensions import TypedDict, Literal
from langgraph.graph import StateGraph, END, START
from langgraph.types import interrupt, Command
from langgraph.types import RunnableConfig
from dataclasses import dataclass

In [3]:
class State(TypedDict):
    user_id: str
    verified: bool
    step: str


2. Build the subgraph (user verification)

In [4]:
def verify_email(state: State) -> dict:
    print(f"Verifying email for {state['user_id']}")
    return {"step": "email_verified"}

def verify_otp(state: State) -> dict:
    print(f"OTP verified for {state['user_id']}")
    return {"step": "otp_verified", "verified": True}

def done(state: State):
    # this node will now be interrupted, and the parent graph will handle the next step
    print("Subgraph done, returning to parent...")
    return

subgraph_builder = StateGraph(State)
subgraph_builder.add_node("verify_email", verify_email)
subgraph_builder.add_node("verify_otp", verify_otp)
subgraph_builder.add_node("done", done)
subgraph_builder.set_entry_point("verify_email")
subgraph_builder.add_edge("verify_email", "verify_otp")
subgraph_builder.add_edge("verify_otp", "done")
subgraph = subgraph_builder.compile(interrupt_after=["done"])

3. Build the parent graph

In [5]:
def router_node(state: State) -> dict:
    print("Routing into subgraph...")
    return {"step": "user_verification"}

def final_node(state: State) -> dict:
    print("Back in parent graph!")
    return {"step": "completed"}

parent_builder = StateGraph(State)
parent_builder.add_node("router_node", router_node)
parent_builder.add_node("user_verification", subgraph)
parent_builder.add_node("final_node", final_node)

parent_builder.add_conditional_edges(
    "router_node",
    lambda x: x["step"],
    {
        "user_verification": "user_verification",
        "final_node": "final_node"
    }
)
parent_builder.add_edge("user_verification", "final_node")
parent_builder.set_entry_point("router_node")
parent_builder.set_finish_point("final_node")
graph = parent_builder.compile()

In [6]:
output = graph.invoke({"user_id": "abc123", "verified": False, "step": ""})
print("Final output:", output)


Routing into subgraph...
Verifying email for abc123
OTP verified for abc123
Subgraph done, returning to parent...
Final output: {'user_id': 'abc123', 'verified': False, 'step': 'user_verification'}
